In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import re
import os
import time
from shapely.geometry import Point, Polygon, LineString,MultiPolygon
from shapely.ops import unary_union

In [2]:
# Reading in the area-separating-indices created in 2.AreaBasedWarnings.ipynb

directory = 'AreaIndices/'
i = 0
areaIndices = {}
for filename in os.listdir(directory):
    with open(directory+filename,'r') as textfile:
        obs = textfile.readlines()
        obs = [int(re.sub('\n','',i)) for i in obs]
        name = re.sub('.txt','',filename)
        areaIndices[name] = obs
worldSeas = gpd.read_file('World_Seas_IHO_v3/World_Seas_IHO_v3.shp')

In [3]:
# Reading in the cleaned data
messages = pd.read_csv('CleanedData.csv',index_col=0)
geoMessages = gpd.GeoDataFrame(messages)

# Making sure that the dataframe eventually will have a 'valid' geometry column 
geometryList = []
for i,(shape,geom) in enumerate(zip(geoMessages.GeometryType,geoMessages.Coordinates)):
    
    if shape == 'Point':
        coor = []
        ele = geom.split()
        ele = [float(re.sub('[](),[]','',i)) for i in ele]

        coor.append(tuple(ele))
        geo = Point(coor)
        geometryList.append(geo)
        
    elif shape == 'GeometryCollection':
        
        coor = []
        ele = geom.split()
        ele = [float(re.sub('[](),[]','',i)) for i in ele]
        
        tupleCoor = [tuple(ele[i:i+3]) for i in np.arange(0,len(ele),3)]
        if i == 4956 or i == 5681 or i == 1037:
            # Necessary because the three observations are LineStrings, or closer to linestrings than polygons.
            geo = LineString(tupleCoor)
        else:
            geo = Polygon(tupleCoor)
        geometryList.append(geo)

In [7]:
# Storing the geometries in something we can work with.
geometryFrame = gpd.GeoDataFrame(geometry=geometryList,crs=worldSeas.crs)

# Storing the centroids
geometryFrame['Centroids']=geometryFrame.centroid
geometryFrame['CentroidCoor'] = [list(geometryFrame['Centroids'].loc[obs].coords)[0] for obs in geometryFrame.index]
# Creating a reduced dataframe based on the unique coordinates
reducedCentroidCoordinates = geometryFrame.drop_duplicates(subset=['CentroidCoor'])

# Creating the reduced geometry and message dataframe
reducedGeometryFrame = geometryFrame.loc[reducedCentroidCoordinates.index]
reducedGeometryFrame = reducedGeometryFrame.reset_index(drop=True)
reducedGeoMessages = geoMessages.loc[reducedCentroidCoordinates.index]
reducedGeoMessages = reducedGeoMessages.reset_index(drop=True)
# Setting the geometry of the reduced-messages-dataframe
reducedGeoMessages['geometry'] = reducedGeometryFrame['geometry']
reducedGeoMessages['Centroids'] = reducedGeometryFrame['Centroids']
reducedGeoMessages['CentroidCoor_X'] = [list(reducedGeoMessages['Centroids'].loc[obs].coords)[0][0] for obs in reducedGeoMessages.index]
reducedGeoMessages['CentroidCoor_Y'] = [list(reducedGeoMessages['Centroids'].loc[obs].coords)[0][1] for obs in reducedGeoMessages.index]
reducedGeoMessages.geometry = reducedGeoMessages['geometry']
reducedGeoMessages.crs = worldSeas.crs
print('This is the areas imported',areaIndices.keys())

This is the areas imported dict_keys(['hydroArc', 'hydroLant', 'hydroPac', 'navareaIV', 'navareaXII'])


In [8]:
avaliableAreas = list(areaIndices.keys())
area0 = reducedGeoMessages.loc[areaIndices[avaliableAreas[0]]]
area1 = reducedGeoMessages.loc[areaIndices[avaliableAreas[1]]]
area2 = reducedGeoMessages.loc[areaIndices[avaliableAreas[2]]]
area3 = reducedGeoMessages.loc[areaIndices[avaliableAreas[3]]]
area4 = reducedGeoMessages.loc[areaIndices[avaliableAreas[4]]]

In [9]:
area0

,Area,Chart,Message,Authority,Date,StartDate,EndDate,BroadcastingDate,GeometryType,geometry,Coordinates,Centroids,CentroidCoor_X,CentroidCoor_Y
0,CANADA-NORTH COAST. /BEAUFORT SEA.,15940,BOTTOM SCIENTIFIC MOORING EXTENDING 3.5 METERS...,NAVAREA XVII,No date listed,2018-06-01 00:00:00,2018-06-01 00:00:00,2018-06-01,Point,POINT Z (-127.688834 70.55800000000001 0),"[(-127.688834, 70.558, 0.0)]",POINT (-127.688834 70.55800000000001),-127.688834,70.558000
18,BEAUFORT SEA./CANADA-NORTH COAST./DNC 27.,15927.,"SCIENTIFIC MOORING, TOP FLOAT 38 METERS BELOW ...",NAVAREA XVII,2017-08-04 00:00:00,2018-06-01 00:00:00,2018-06-01 00:00:00,2018-06-01,Point,POINT Z (-127.660334 70.576667 0),"[(-127.660334, 70.576667, 0.0)]",POINT (-127.660334 70.576667),-127.660334,70.576667
19,AMUNDSEN GULF./CANADA-NORTH COAST./DNC 27.,15927.,"SCIENTIFIC MOORING, TOP FLOAT 292 METERS BELOW...",NAVAREA XVII,2017-08-04 00:00:00,2018-06-01 00:00:00,2018-06-01 00:00:00,2018-06-01,Point,POINT Z (-126.871334 70.681167 0),"[(-126.871334, 70.681167, 0.0)]",POINT (-126.871334 70.681167),-126.871334,70.681167
20,CANADA./DNC 28.,15881.,SHOALS REPORTED: A. 6.0 METERS IN 69-30.76N 09...,NAVAREA XVIII,2017-08-26 00:00:00,2018-06-01 00:00:00,2018-06-01 00:00:00,2018-06-01,Point,POINT Z (-94.984167 69.51266699999999 0),"[(-94.984167, 69.512667, 0.0)]",POINT (-94.984167 69.51266699999999),-94.984167,69.512667
21,CANADA./DNC 28.,15881.,SHOALS REPORTED: A. 6.0 METERS IN 69-30.76N 09...,NAVAREA XVIII,2017-08-26 00:00:00,2018-06-01 00:00:00,2018-06-01 00:00:00,2018-06-01,Point,POINT Z (-94.969167 69.51600000000001 0),"[(-94.969167, 69.516, 0.0)]",POINT (-94.969167 69.51600000000001),-94.969167,69.516000
22,CANADA./DNC 28.,15881.,SHOALS REPORTED: A. 6.0 METERS IN 69-30.76N 09...,NAVAREA XVIII,2017-08-26 00:00:00,2018-06-01 00:00:00,2018-06-01 00:00:00,2018-06-01,Point,POINT Z (-94.9755 69.513834 0),"[(-94.9755, 69.513834, 0.0)]",POINT (-94.9755 69.513834),-94.975500,69.513834
23,CANADA. /DNC 28.,15881.,4.27 METER DEPTH REPORTED IN 68-24.82N 096-21....,NAVAREA XVIII,2017-09-12 00:00:00,2018-06-01 00:00:00,2018-06-01 00:00:00,2018-06-01,Point,POINT Z (-96.360834 68.413667 0),"[(-96.360834, 68.413667, 0.0)]",POINT (-96.360834 68.413667),-96.360834,68.413667
31,CORONATION GULF./CANADA/DNC 28.,15955.,"SUB-SURFACE SCIENTIFIC MOORING, TOP FLOAT 40 M...",NAVAREA XVIII,2017-09-21 00:00:00,2018-06-01 00:00:00,2018-06-01 00:00:00,2018-06-01,Point,POINT Z (-113.388834 68.46083400000001 0),"[(-113.388834, 68.460834, 0.0)]",POINT (-113.388834 68.46083400000001),-113.388834,68.460834
40,BEAUFORT SEA./AMUNDSEN GULF./CANADA./DNC 27.,15927.,SCIENTIFIC MOORINGS ESTABLISHED IN: A. 71-18.6...,NAVAREA XVII,2017-09-28 00:00:00,2018-06-01 00:00:00,2018-06-01 00:00:00,2018-06-01,Point,POINT Z (-126.028167 70.988167 0),"[(-126.028167, 70.988167, 0.0)]",POINT (-126.028167 70.988167),-126.028167,70.988167
2095,LANCASTER SOUND./CANADA./DNC 28.,NO CHART,"M/V NATIONAL GEOGRAPHIC EXPLORER, OVERDUE IN ...",JRCC TRENTON,2018-08-13 00:00:00,2018-08-14 00:00:00,2018-08-14 00:00:00,2018-08-14,Point,POINT Z (-82.383334 74.516667 0),"[(-82.383334, 74.516667, 0.0)]",POINT (-82.383334 74.516667),-82.383334,74.516667


If one needs to work with all the data, the below is necessary to execute, otherwise just use the above.

In [8]:
# Updating the georeferenced messages with correct geometries
geoMessages['geometry'] = geometryFrame['geometry']
geoMessages['Centroids'] = geometryFrame['Centroids']
geoMessages.geometry = geoMessages['geometry']
geoMessages.crs = worldSeas.crs

# Creating a column in the full and the reduced datasset, which is mergable.
#geometryFrame['CentroidCoor'] = [list(geometryFrame['Centroids'].loc[obs].coords)[0] for obs in geometryFrame.index]
reducedGeometryFrame['CentroidCoor'] = [list(reducedGeometryFrame['Centroids'].loc[obs].coords)[0] for obs in reducedGeometryFrame.index]

# Extracting all the subarea-tags
area = []
for obs in reducedGeometryFrame.index:
    area.append([i for i,subarea in enumerate(areaIndices.keys()) if obs in areaIndices[subarea]][0])
reducedGeometryFrame['Subarea'] = area

# Merging the reduced dataset, with subarea-tags, onto the fulldataset.
mergedCentroids = geometryFrame.merge(reducedGeometryFrame,how='left',on=['CentroidCoor'])

In [10]:
subareasMessages = {subarea : mergedCentroids[mergedCentroids['Subarea']==i].shape[0] for i,subarea in enumerate(areaIndices.keys())}

In [11]:
subareasMessages

{'hydroArc': 6194,
 'hydroLant': 3145,
 'hydroPac': 12455,
 'navareaIV': 25593,
 'navareaXII': 6569}

In [14]:
mergedCentroids


,geometry_x,Centroids_x,CentroidCoor,geometry_y,Centroids_y,Subarea
0,POINT Z (-127.688834 70.55800000000001 0),POINT (-127.688834 70.55800000000001),"(-127.688834, 70.558)",POINT Z (-127.688834 70.55800000000001 0),POINT (-127.688834 70.55800000000001),0
1,POINT Z (-133.714 70.0585 0),POINT (-133.714 70.0585),"(-133.714, 70.0585)",POINT Z (-133.714 70.0585 0),POINT (-133.714 70.0585),0
2,POINT Z (-133.717167 70.0585 0),POINT (-133.717167 70.0585),"(-133.717167, 70.0585)",POINT Z (-133.717167 70.0585 0),POINT (-133.717167 70.0585),0
3,POINT Z (-158.410434 72.7996 0),POINT (-158.410434 72.7996),"(-158.410434, 72.7996)",POINT Z (-158.410434 72.7996 0),POINT (-158.410434 72.7996),0
4,POINT Z (-158.702167 72.615167 0),POINT (-158.702167 72.615167),"(-158.702167, 72.615167)",POINT Z (-158.702167 72.615167 0),POINT (-158.702167 72.615167),0
5,POINT Z (-158.412667 72.800167 0),POINT (-158.412667 72.800167),"(-158.412667, 72.800167)",POINT Z (-158.412667 72.800167 0),POINT (-158.412667 72.800167),0
6,POINT Z (-139.020667 70.432334 0),POINT (-139.020667 70.432334),"(-139.020667, 70.432334)",POINT Z (-139.020667 70.432334 0),POINT (-139.020667 70.432334),0
7,POINT Z (-135.011 70.86966700000001 0),POINT (-135.011 70.86966700000001),"(-135.011, 70.869667)",POINT Z (-135.011 70.86966700000001 0),POINT (-135.011 70.86966700000001),0
8,POINT Z (-135.019167 70.869 0),POINT (-135.019167 70.869),"(-135.019167, 70.869)",POINT Z (-135.019167 70.869 0),POINT (-135.019167 70.869),0
9,POINT Z (-161.500184 71.600117 0),POINT (-161.500184 71.600117),"(-161.500184, 71.600117)",POINT Z (-161.500184 71.600117 0),POINT (-161.500184 71.600117),0
